# Training Notebook

## Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import cv2
import albumentations as A
import torchvision.models as models
import os
from tqdm.notebook import tqdm
import time

## Resize and save the image

In [21]:
def resize_img(input_path, write_path, size=(256, 256)):
    img = cv2.imread(input_path)
    img = cv2.resize(img, size)
    cv2.imwrite(write_path, img)

## Augmentations pipeline

In [22]:
# Define transformations
transforms = A.Compose([
    A.Flip(p=0.5),
    A.Rotate(limit=10, 
             border_mode=cv2.BORDER_CONSTANT, 
             value=0.0, p=0.75),
    A.RandomResizedCrop(width=224, height=224, scale=(0.33, 1), p=1)
])

## Dataset

In [6]:
class ArtDataset(Dataset):
    def __init__(self, df, label_dict, transforms=False):
        self.df = df
        self.transforms = transforms
        self.label_dict = label_dict
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Get filename and label
        filename = ...
        label = torch.LongTensor(self.label_dict[row['label']])
        # Read image, correct color channels
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Augmentations
        if self.transforms:
            transformed = self.transforms(image=img.astype(np.uint8))
            img = transformed['image']
        # Convert to array, switch channels, scale, return
        # img = np.transpose(img, (2, 0, 1))
        img = torch.tensor(img / 255.).float()
        return img, label

## Transfer learning functions

In [16]:
def set_classification_layer(model, model_type='vgg', num_classes=25):
    if model_type == 'vgg':
        model.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )
    elif model_type == 'resnet':
        model.fc = nn.Linear(in_features=4096, out_features=num_classes, bias=True)
    elif model_type == 'vit':
        model.heads = nn.Linear(in_features=768, out_features=num_classes, bias=True)
    elif model_type == 'convnext':
        model.classifier = nn.Sequential(
            nn.LayerNorm2d((768,), eps=1e-06, elementwise_affine=True),
            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(in_features=768, out_features=num_classes, bias=True)
        )
    else:
        print(f'Unknown model_type {model_type}. Acceptable types are: "vgg", "resnet", "vit", or "convnext"')
    

In [17]:
def freeze_model(model, **classargs):
    '''
    Given an existing model, freeze pre-trained weights and
    re-instantiate the classifier.
    '''
    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    # Re-instantiate the classifier head
    set_classification_layer(model, **classargs)

In [18]:
vgg = models.vgg16()
for name, param in vgg.named_parameters():
    print(f"{name} gradient is set to", param.requires_grad)

features.0.weight gradient is set to True
features.0.bias gradient is set to True
features.2.weight gradient is set to True
features.2.bias gradient is set to True
features.5.weight gradient is set to True
features.5.bias gradient is set to True
features.7.weight gradient is set to True
features.7.bias gradient is set to True
features.10.weight gradient is set to True
features.10.bias gradient is set to True
features.12.weight gradient is set to True
features.12.bias gradient is set to True
features.14.weight gradient is set to True
features.14.bias gradient is set to True
features.17.weight gradient is set to True
features.17.bias gradient is set to True
features.19.weight gradient is set to True
features.19.bias gradient is set to True
features.21.weight gradient is set to True
features.21.bias gradient is set to True
features.24.weight gradient is set to True
features.24.bias gradient is set to True
features.26.weight gradient is set to True
features.26.bias gradient is set to True


In [19]:
# Freeze model
freeze_model(vgg, num_classes=25, model_type='vgg')
vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=25, bias=True)
)

In [20]:
for name, param in vgg.named_parameters():
    print(f"{name} gradient is set to", param.requires_grad)

features.0.weight gradient is set to False
features.0.bias gradient is set to False
features.2.weight gradient is set to False
features.2.bias gradient is set to False
features.5.weight gradient is set to False
features.5.bias gradient is set to False
features.7.weight gradient is set to False
features.7.bias gradient is set to False
features.10.weight gradient is set to False
features.10.bias gradient is set to False
features.12.weight gradient is set to False
features.12.bias gradient is set to False
features.14.weight gradient is set to False
features.14.bias gradient is set to False
features.17.weight gradient is set to False
features.17.bias gradient is set to False
features.19.weight gradient is set to False
features.19.bias gradient is set to False
features.21.weight gradient is set to False
features.21.bias gradient is set to False
features.24.weight gradient is set to False
features.24.bias gradient is set to False
features.26.weight gradient is set to False
features.26.bias g

## Training functions

In [ ]:
# Dictionary for easily passing training arguments
training_params = {'epochs': 10,
                  'batch_size': 16,
                  'loss_fct': nn.CrossEntropyLoss()}

In [ ]:
def eval_model(model, dl, training_params):
    # Get GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # Evaluate
    model.eval()
    total_loss = 0
    total_obs = 0
    total_correct = 0
    loss_fct = training_params['loss_fct']
    for X, y in dl:
        n_obs = len(y)
        # Forward pass and calculate loss
        yhat = model(X.to(device))#.softmax(dim=1)
        loss = loss_fct(yhat.to(device), y.to(device))
        total_loss += n_obs * loss.item()
        total_obs += n_obs
        # Calculate batch accuracy
        ypred = np.argmax(yhat.cpu().detach().numpy(), axis=1)
        y_arr = y.detach().numpy()
        total_correct += n_obs * accuracy_score(y_arr, ypred)
    # Return loss, accuracy
    avg_loss = total_loss / total_obs
    accuracy = total_correct / total_obs
    return avg_loss, accuracy
    
    
def train_model(model, optimizer, scheduler, train_ds, valid_ds, training_params):
    # Get loss function
    loss_fct = training_params['loss_fct']
    # Create dataloaders based on batch size
    batch_size = trainin_params['batch_size']
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    # Get GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    # Train
    for _ in range(training_params['epochs']):
        # Put model in train mode
        model.train()
        # Train on training dataloader
        for X, y in train_dl:
            # Clear gradients
            optimizer.zero_grad()
            # Forward pass and loss calculation
            yhat = model(X.to(device))#.softmax(dim=1)
            loss = loss_fct(yhat.to(device), y.to(device))
            # Backward pass and step
            loss.backward()
            optimizer.step()
            scheduler.step()
        # Calculate loss, accuracy on train and validation
        train_loss, train_acc = eval_model(model, train_dl)
        valid_loss, valid_acc = eval_model(model, valid_dl)
        train_str = f"train loss: {train_loss:.4f} | train acc: {train_acc:.4f}"
        valid_str = f" | valid loss: {valid_loss:.4f} | valid acc: {valid_acc:.4f}"
        print(f'[{_}] ' + train_str + valid_str)
    

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'